In [1]:
import pandas as pd
import tweepy
import requests
import datetime
import json
import matplotlib.pyplot as plt
from IPython.display import display
import time
import random
from threading import Thread
from tqdm import tqdm, trange
import multiprocessing as mp
from  datetime import datetime
from Classes import *
import re
import os
from functions import *
import torch
import numpy as np

In [2]:
f = 'Biden/2022-11-07/user_profile/Profile.xlsx'
states = list()
with open('Biden/2022-11-07/user_profile/US_States_Cities.txt', 'r') as txt:
    lines = txt.readlines()
    for line in lines:
        s = line.replace('\n', '')
        states.append(s)
    
df = pd.read_excel(f, sheet_name='profile')
df['class'] = -1
df.loc[df['location']=='None', 'class'] = 0


In [3]:
df

,id,location,followers_count,following_count,tweet_count,verified,created_at,class
0,56881451,None,0,56,103,False,2009-07-15 01:22:12+00:00,0
1,348399706,Saudi Arabia Hail Live,0,1583,106372,False,2011-08-04 11:02:11+00:00,-1
2,4771801823,None,0,68,2181,False,2016-01-11 06:50:46+00:00,0
3,1331361875800502274,None,0,214,9978,False,2020-11-24 22:20:02+00:00,0
4,1336815260775317510,Afghanistan,0,40,52,False,2020-12-09 23:29:41+00:00,-1
...,...,...,...,...,...,...,...,...
12556,818893114979061761,"New Jersey, USA",711195,4728,146783,False,2017-01-10 18:51:47+00:00,-1
12557,307549658,"Miami, FL",812727,8999,81995,False,2011-05-29 20:59:40+00:00,-1
12558,2573480784,"Philadelphia PA, Charleston SC",1156064,2602,250308,True,2014-06-17 20:09:43+00:00,-1
12559,27493883,"New York, Florida",1168579,69056,368527,True,2009-03-29 20:12:52+00:00,-1


In [4]:
states = [state.lower() for state in states]
progress = tqdm(total = len(df.index))
for index in df.index:
    loc = df.loc[index, 'location']
    c = df.loc[index, 'class']
    if c == -1:
        for state in states:
            if state in loc.lower() or loc.lower() in state:
                df.loc[index, 'class'] = 1
            elif 'united' in loc.lower() or 'usa' in loc.lower():
                df.loc[index, 'class'] = 1
        progress.update(1)

 56%|█████▌    | 7048/12561 [09:06<07:22, 12.47it/s]

In [ ]:
states

In [8]:
df[df['class'] == -1]

,id,location,followers_count,following_count,tweet_count,verified,created_at,class
1,348399706,Saudi Arabia Hail Live,0,1583,106372,False,2011-08-04 11:02:11+00:00,-1
4,1336815260775317510,Afghanistan,0,40,52,False,2020-12-09 23:29:41+00:00,-1
7,1504007843053195265,"Umeå, Sverige",0,153,24603,False,2022-03-16 08:44:43+00:00,-1
11,1567916383329034250,Sardon de duero,0,96,32666,False,2022-09-08 16:43:11+00:00,-1
15,1584511067815907328,"West Bengal, India",0,118,1714,False,2022-10-24 11:44:31+00:00,-1
...,...,...,...,...,...,...,...,...
12534,103161101,Planeta Tierra,106402,97512,251514,False,2010-01-09 03:08:38+00:00,-1
12536,65675400,Selena Protection Program,119851,18578,920765,False,2009-08-14 16:38:45+00:00,-1
12539,49532208,Worldwide,154116,78465,283003,False,2009-06-22 04:09:36+00:00,-1
12542,2917514307,Tanzania,175437,130445,698140,False,2014-12-12 13:04:34+00:00,-1


In [9]:
df[df['class'] == 1]

,id,location,followers_count,following_count,tweet_count,verified,created_at,class
18,166670568,New York,1,30,137,False,2010-07-14 18:44:42+00:00,1
28,1568361730832490498,"Brooklyn, NY",1,180,164,False,2022-09-09 22:13:27+00:00,1
31,1575969988254568454,"California, USA",1,0,800,False,2022-09-30 22:05:13+00:00,1
36,1585015209340567552,"California, USA",1,0,108,False,2022-10-25 21:07:47+00:00,1
41,1587150315152736256,"Queens,NY",1,24,119,False,2022-10-31 18:32:44+00:00,1
...,...,...,...,...,...,...,...,...
12556,818893114979061761,"New Jersey, USA",711195,4728,146783,False,2017-01-10 18:51:47+00:00,1
12557,307549658,"Miami, FL",812727,8999,81995,False,2011-05-29 20:59:40+00:00,1
12558,2573480784,"Philadelphia PA, Charleston SC",1156064,2602,250308,True,2014-06-17 20:09:43+00:00,1
12559,27493883,"New York, Florida",1168579,69056,368527,True,2009-03-29 20:12:52+00:00,1


In [10]:
df[df['class'] == 0]

,id,location,followers_count,following_count,tweet_count,verified,created_at,class
0,56881451,None,0,56,103,False,2009-07-15 01:22:12+00:00,0
2,4771801823,None,0,68,2181,False,2016-01-11 06:50:46+00:00,0
3,1331361875800502274,None,0,214,9978,False,2020-11-24 22:20:02+00:00,0
5,1431739673647407164,None,0,58,702,False,2021-08-28 22:06:37+00:00,0
6,1446110202034413606,None,0,22,9,False,2021-10-07 13:50:08+00:00,0
...,...,...,...,...,...,...,...,...
12530,15554922,None,92297,7360,80935,True,2008-07-23 03:36:34+00:00,0
12543,1351293685493878786,None,180171,68,1919,True,2021-01-18 22:22:12+00:00,0
12545,40996128,None,195942,1828,108343,True,2009-05-18 23:14:51+00:00,0
12546,467726691,None,228201,4175,419169,True,2012-01-18 19:44:35+00:00,0


In [11]:
df.to_excel(f, sheet_name='profile')

In [46]:
file_path = 'Biden/2022-11-07/user_profile/'
to_excel = dict()
for i in range(14):
    with open(file_path+'%d.json'%i, 'r') as f:
        json_obj = json.load(f)
        for key in json_obj.keys():
            if json_obj[key] == None:
                continue
            else:
                to_excel[key] = json_obj[key]

to_excel = pd.DataFrame.from_dict(to_excel, orient='index')
to_excel['followers_count'].astype('str')
to_excel['following_count'].astype('str')
to_excel['tweet_count'].astype('str')
#to_excel.to_excel(file_path + 'Profile.xlsx', sheet_name='profile')
to_excel.reset_index(inplace=True)
to_excel.rename( columns={ 'index':'id'}, inplace=True )
len(to_excel['id'].unique())

12561

In [ ]:
User_Profile(
    user=[704863841092505600],
    token = 'AAAAAAAAAAAAAAAAAAAAAPwxjQEAAAAAUG%2FmQd1X42iX%2FYsqWpOPk97RyAo%3DmWe7RLkTyX7mxP0K6MhOX6g96fJrdLP2pgffm6cHLEooRKTzao',
    index=15,
    exist=set([]),
    file_path=''
)

In [ ]:
file_path = 'Biden/2022-11-07/user_tweets/'
key = '9624742'
count = 0
for i in range(14):
    if path.exists(file_path + "%d.txt"%i):
        with open(file_path + "%d.txt"%i, 'r') as f:
            lines = f.readlines()
            for line in lines:
                s = line.split(' ')
                if key == s[0] and s[2] != 'None':
                    count += 1

count

In [ ]:
path = 'Biden/2022-11-07/retweeters/'
files = os.listdir(path)
files = sorted(files)

key = 9624742
count = 0
for file in files:
    json_data = dict()
    with open(path + file, 'r') as f:
        json_data = json.load(f)
        for i in json_data.keys():
            if key in json_data[i]:
                count += 1

count

In [ ]:
print(datetime.datetime.now())
exist = set()
file_path = 'Biden/2022-11-07/followers/'
for i in range(14):
            # print(i)
    if path.exists(file_path + "%d_followers.json" % i):
        tmp = set()
        with open(file_path + "%d_followers.json" % i, "r") as f:
            file = json.load(f)
            for key in list(file.keys()):
                tmp.add(key)
                exist.add(key)
            print("%d_followers.json :" % i, len(tmp))
        
print("Already crawled :", len(exist))

In [ ]:
file_path  = 'Biden/2022-11-07/user_tweets/'
exist = set()
if path.exists(file_path):
            for i in range(14):
                if path.exists(file_path + '%d.json'%i): # check files exist or not
                    with open(file_path + '%d.json'%i, 'r') as f:
                        json_file = json.load(f)
                        for key in json_file.keys(): # key is target user's id
                            exist.add(int(key))

            for i in range(14):
                if path.exists(file_path + '%d.txt'%i):
                    with open(file_path + '%d.txt'%i, 'r') as f:
                        lines = f.readlines()
                        for line in lines:
                            if '=' not in line:
                                s = line.split(' ')
                                exist.add(int(s[0]))
            exist = list(exist)
            print("Already crawled :", len(exist))

In [ ]:
print(datetime.datetime.now())
p = 'Biden/2022-11-07/user_tweets/'
files = os.listdir(p)
txt_files = []
txt_ele = set()
for i in files:
    if '.txt' in i and 'Biden' not in i and 'unfinish' not in i: 
        txt_files.append(i)
txt_files = sorted(txt_files)

for file in range(len(txt_files)):
    tmp = set()
    with open(p + "%d.txt"%file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            s = line.split(' ')
            txt_ele.add(s[0])
            tmp.add(s[0])
        print("%d.txt :"%file, len(tmp))
print("total :", len(txt_ele))

In [ ]:
print(datetime.datetime.now())
p = 'Biden/2022-11-07/user_tweets/'
files = os.listdir(p)
txt_files = []
txt_ele = set()
for i in files:
    if '.txt' in i and 'Biden' not in i and 'unfinish' not in i: 
        txt_files.append(i)
txt_files = sorted(txt_files)

for file in range(len(txt_files)):
    tmp = set()
    with open(p + "%d.txt"%file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            s = line.split(' ')
            txt_ele.add(s[0])
            tmp.add(s[0])
        print("%d.txt :"%file, len(tmp))
print("total :", len(txt_ele))

In [ ]:
print(datetime.datetime.now())
p = 'Biden/2022-11-07/user_tweets/'
files = os.listdir(p)
json_file  = []
for i in files:
    if '.json' in i and 'Biden' not in i: 
        json_file.append(i)
json_file = sorted(json_file)

count = 0
for i in json_file:
    with open(p+i, 'r') as f:
        json_data = json.load(f)
        for key in json_data.keys():
            txt_ele.add(s[0])
print("No retweet :", len(txt_ele))

In [ ]:
dir_path = 'Biden/2022-10-10'
f = list()
l = list()
u_t = dict()
ids = set()
for p in os.listdir(dir_path):
    if ".txt" in p:
        f.append(p)
f.remove('exist.txt')
f = sorted(f)

In [ ]:
for file in f:
    file = "/" + file
    with open(dir_path + file, "r") as g:
        for line in g:
            if "=" in line:
                continue
            
            s = line.replace("\n", "")
            s = s.split(' ')
            ids.add(s[0])
u_t = u_t.fromkeys(ids, [])


In [ ]:
for file in f:
    file = "/" + file
    with open(dir_path + file, "r") as g:
        value = list()
        for line in g:
            
            s = line.replace("\n", "")
            if "=" in s:
                u_t[key] = value
                value = []
            
            else:
                
                s = s.split(' ')
                key = s[0]
                ref = s[1]
                tweet_id = s[2]
                create_at = s[3] + " " + s[4]
                tmp = {
                    "ref":ref, 
                    "tweet_id" : tweet_id, 
                    "create":create_at
                }
                value.append(tmp)

In [ ]:
with open(dir_path + "/user_tweets.json", "w") as f:
    json.dump(u_t, f, indent=True)

In [ ]:
a = list(u_t.keys())
a = a[0]
len(u_t[a])

In [ ]:
for  i in u_t.keys():
    print(len(u_t[i]))

In [ ]:
print(datetime.datetime.now())
i = 0
path = "Biden/2022-10-10/"
summ = list()
for i in range(8):
    with open("Biden/2022-10-10/%d_followers.json" % i, "r") as f:
        a = json.load(f)
        summ.extend(list(a.keys()))
        print("%d :" %i , len(a.keys()))
        #print(len(a))
print(len(summ))

In [ ]:
s = "<ReferencedTweet id=1579540287097294850 type=replied_to"
x = re.search("id=", s).end()
integers = [str(i) for i in range(10)]
id = ''
for i in range(x, len(s)):
    if s[i] == ' ':
        break
    if s[i] in integers:
        id += s[i]
id    

In [ ]:
dic = dict()
dic1 = dict()
dic2 = dict()
l = list()

f = open('0.json', 'w')
for i in range(3):
    k = [j for j in range(10)]
    l.append(k)

json.dump(l, f, indent=4)
f.close()



In [ ]:
token = [
'AAAAAAAAAAAAAAAAAAAAAIPvbAEAAAAApt3kcfjyiVBOLVWMbi1Fu6CLL24%3DuIMwpFYplQqMd68VXXv4eU954tnOQu5vljRnV3xL292k4F5sFR',
'AAAAAAAAAAAAAAAAAAAAACHvcgEAAAAA9lDgALcD%2FZRxz%2BFrzuc6B7IxJqc%3DnBTsObUCrmc50dlBlB81QMnqnab8oj5Oo7uy4uXMi7vVUOEJAu',
'AAAAAAAAAAAAAAAAAAAAAIPvbAEAAAAApt3kcfjyiVBOLVWMbi1Fu6CLL24%3DuIMwpFYplQqMd68VXXv4eU954tnOQu5vljRnV3xL292k4F5sFR',
'AAAAAAAAAAAAAAAAAAAAAOvxbAEAAAAAOu%2FTOVHN5ocKWaChbnjZwI%2FoGdw%3DC1laLqLeCuXiwjg1bkV7kEVMY3pd3kZHDA4TKeRjcR89kKQPeR',
'AAAAAAAAAAAAAAAAAAAAAC3ucgEAAAAANkLeJqeojqdGXtbsGknDrLiZuJ4%3DueRC8NneqPUawhC3rkI83DXgUk68poiPHc60YAMtKqDr9NFTHs',
'AAAAAAAAAAAAAAAAAAAAAHfucgEAAAAAWrhXRgMpo%2Fj7fHl9uqSiUlklY5M%3DQWmApmhHTWG0LcdB77fiYOTnB8mbpU4HcPLMeWl6mQmZr60mKZ',
'AAAAAAAAAAAAAAAAAAAAAJKnJgEAAAAAgpYwO66pBtdDUcwjHYnWbMY1ow8%3DJgCTSnmkqVE5M0ywnyoSvMX1Ywpz2ivYnONwQ0Aro7bgby6TUm',
'AAAAAAAAAAAAAAAAAAAAAJaOeQEAAAAAyvSZWQ6m9q4SaMT1t1WCH%2FOymj0%3DVgRvLEP51YERK403i2pk5JU05Lu0vXppgQHxvhhxcBIkftu1FH'
]

In [ ]:
pro = list()
for i in range(4):
    p = mp.Process(target=h, args=(i, ))
    pro.append(p)
for i in iter(pro):
    i.start()
for i in iter(pro):
    i.join()

In [ ]:
file_path = ''
data = Data_crawl(
    token=token
)
data.Get_Followers_MultiCore()

In [ ]:
today = '2022-10-08'  #filename = str(datetime.date.today()) # 用當天日期當作檔案名稱
filename = 'Biden/' + today + '/' + today
filename = filename + '_' + str(0) + '_retweeters.json'
filename

必要 parameters

In [ ]:
token = [
'AAAAAAAAAAAAAAAAAAAAACHvcgEAAAAA9lDgALcD%2FZRxz%2BFrzuc6B7IxJqc%3DnBTsObUCrmc50dlBlB81QMnqnab8oj5Oo7uy4uXMi7vVUOEJAu',
'AAAAAAAAAAAAAAAAAAAAAIPvbAEAAAAApt3kcfjyiVBOLVWMbi1Fu6CLL24%3DuIMwpFYplQqMd68VXXv4eU954tnOQu5vljRnV3xL292k4F5sFR',
'AAAAAAAAAAAAAAAAAAAAAOvxbAEAAAAAOu%2FTOVHN5ocKWaChbnjZwI%2FoGdw%3DC1laLqLeCuXiwjg1bkV7kEVMY3pd3kZHDA4TKeRjcR89kKQPeR',
'AAAAAAAAAAAAAAAAAAAAAC3ucgEAAAAANkLeJqeojqdGXtbsGknDrLiZuJ4%3DueRC8NneqPUawhC3rkI83DXgUk68poiPHc60YAMtKqDr9NFTHs',
'AAAAAAAAAAAAAAAAAAAAAHfucgEAAAAAWrhXRgMpo%2Fj7fHl9uqSiUlklY5M%3DQWmApmhHTWG0LcdB77fiYOTnB8mbpU4HcPLMeWl6mQmZr60mKZ',
'AAAAAAAAAAAAAAAAAAAAAJKnJgEAAAAAgpYwO66pBtdDUcwjHYnWbMY1ow8%3DJgCTSnmkqVE5M0ywnyoSvMX1Ywpz2ivYnONwQ0Aro7bgby6TUm',
'AAAAAAAAAAAAAAAAAAAAAJaOeQEAAAAAyvSZWQ6m9q4SaMT1t1WCH%2FOymj0%3DVgRvLEP51YERK403i2pk5JU05Lu0vXppgQHxvhhxcBIkftu1FH'
]

# 時間格式需要符合 twitter 的格式：Year-Month-date + "T" + Hour:Minute:Second + "Z"
end_time  = datetime.datetime.today() # end time is today's 00:00
end_time = end_time.strftime('%Y-%m-%dT')
end_time = end_time + "00:00:00Z"

start_time = datetime.datetime.today() - datetime.timedelta(days= 8) # start time is 7 days ago
start_time = start_time.strftime('%Y-%m-%dT')
start_time = start_time + "00:00:00Z"

In [ ]:
dataset = Data_crawl(token=token)
tweet_df = dataset.Get_Tweets_Dataframe(
    start_time=start_time,
    end_time=end_time
)

In [ ]:
dataset.Get_Retweeters_dict_MultiThread()

In [ ]:
class MyThread(Thread):
    def __init__(self, func, args):
        super(MyThread, self).__init__()
        self.func = func
        self.args = args
        self.result = 0
    def run(self):
        self.result = self.func(*self.args)

    def get_results(self):
        return self.result

In [ ]:
l = [1,2,3,4,5]
l = l[0:2]

In [ ]:
d = {1:2, 2:3, 4:5, 6:9, 3:0}
f = open("2022-10-107_retweeters.json", 'w') 
#t = json.load(f)
t = json.dump(d, f, indent=True)
f.close()

In [ ]:
def g(l:list):
    s = random.randint(3,6)
    print(s, " second")
    time.sleep(s)
    v = {s:l}
    #with open(str(s) + '.json', 'w') as f:
        #a = json.dumps(v)
        #f.write(a)
t = [1,2,3,4,5,6,7,8,9,10]
thread = []
r = []
for i in range(3):
    y = MyThread(func=g, args=(t,))
    thread.append(y)
for i in range(3):
    thread[i].start()
for i in range(3):
    thread[i].join()

r

In [ ]:
t1 = MyThread(func=g, args=(t,))
t2 = MyThread(func=g, args=(t,))
t1.start()
t2.start()
t1.join()
t2.join()
res1 = t1.get_results()
res2 = t2.get_results()
print(res1, res2)

In [ ]:
print(t)

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
a = [i for i in range(55)]
a = chunks(a, int(len(a)/3)+1)
len(list(a))